# Crime Insights London

## Problem Statement

The data is obtained from the [LONDON DATASTORE](https://data.london.gov.uk/dataset). These crimes span from 2010 to 2024.

How have crime rates evolved across different boroughs in London over the past decade, and what are the key factors driving these trends? Can we predict future crime hotspots and periods of increased criminal activity based on historical data, and what strategies could be employed to mitigate these risks?

## Project Goal

The primary goal of this project is to analyse historical crime data to uncover trends, patterns, and correlations across different crime categories and boroughs. 
Additionally, the project aims to develop predictive models to forecast future crime rates and identify potential hotspots. These insights can be leveraged by law enforcement agencies to allocate resources more effectively, enhance community safety, and develop targeted intervention strategies.

## Project Approach

1. Data Pre-processing

2. EDA

3. Trend and Pattern Analysis

4. Predictive Modelling

5. Risk Assessment and Mitigation Strategies

6. Evaluation and Reporting

7. Implementation Roadmap

## Example Use-cases
* Resource Allocation: Use the predictive models to optimise the deployment of police forces and resources in areas with anticipated high crime rates.
* Policy Development: Provide data-driven recommendations to policymakers for developing targeted crime prevention programs.
* Public Awareness: Use trend analysis to inform the public about crime risks in their areas and promote community safety initiatives.


In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [2]:
file_path_bronze = '/Users/joshnolan/programmingProjects/CrimeInsights/LondonCrimeInsights/data/bronze_data/bronze_crime_monthly.csv'
df_bronze_crime = pd.read_csv(file_path_bronze)

In [3]:
# Function for inital data exploration

def analyse_missing_values(df):
    """
    Analyses missing values from df.

    Parameters:
    df (DataFrame): The pandas DataFrame with data.

    Returns:
    Count of rows and columns that have at least one null value and the related percentage.
    """
    missing_rows = df.isnull().any(axis=1).sum()
    missing_columns = df.isnull().any(axis=0).sum()
    total_rows, total_columns = df.shape
    print(f"Missing data points row-wise: {missing_rows} out of {total_rows} ({missing_rows/total_rows*100:.2f}%)")
    print(f"Missing data points column-wise: {missing_columns} out of {total_columns} ({missing_columns/total_columns*100:.2f}%)")

def aggregate_years(df, year_columns, non_time_columns):
    """
    Aggregates monthly data into yearly data.

    Parameters:
    df (DataFrame): The pandas DataFrame with monthly data.
    year_columns (list): List of years to aggregate data for.

    Returns:
    DataFrame: A new DataFrame with data aggregated by year.
    """
    df_yearly = pd.DataFrame()
    for year in year_columns:
        cols_for_year = [col for col in df.columns if col.startswith(year)]
        df_yearly[year] = df[cols_for_year].sum(axis=1)
    df_yearly[non_time_columns] = df[non_time_columns]
    return df_yearly[non_time_columns + sorted(year_columns)]

def detect_outliers(df, numeric_columns):
    """
    Detects outliers in the numeric columns of a DataFrame using the Interquartile Range (IQR) method.

    Parameters:
    df (DataFrame): The pandas DataFrame to analyze for outliers.
    numeric_columns (list): List of column names in df that are numeric.

    Returns:
    tuple: 
        - DataFrame containing only the rows identified as outliers in numeric columns.
        - Series indicating the count of outliers in each numeric column.
    """
    Q1 = df[numeric_columns].quantile(0.25)
    Q3 = df[numeric_columns].quantile(0.75)
    IQR = Q3 - Q1

    outlier_condition = ((df[numeric_columns] < (Q1 - 1.5 * IQR)) | (df[numeric_columns] > (Q3 + 1.5 * IQR)))
    outliers = outlier_condition.any(axis=1)
    outliers_df = df[outliers]
    outliers_count = outlier_condition.sum()

    return outliers_df, outliers_count


In [7]:
# Inspect the merged data
print("Shape:", df_bronze_crime.shape)

display(df_bronze_crime.head())
display(df_bronze_crime.dtypes)
# Missing values analysis
print("Monthly df:")
print(f"Total nulls: {df_bronze_crime.isnull().sum()}")
analyse_missing_values(df_bronze_crime)

# outlier analysis
numeric_cols = df_bronze_crime.select_dtypes(include=['number']).columns.tolist()
outliers_df, outliers_count = detect_outliers(df_bronze_crime, numeric_cols)

# print("Outliers detected in each numeric column:\n", outliers_count)
# print("\nRows with outliers in numeric columns:\n", outliers_df)


Shape: (1746, 169)


,MajorText,MinorText,LookUp_BoroughName,201004,201005,201006,201007,201008,201009,201010,201011,201012,201101,201102,201103,201104,201105,201106,201107,201108,201109,201110,201111,201112,201201,201202,201203,201204,201205,201206,201207,201208,201209,201210,201211,201212,201301,201302,201303,201304,201305,201306,201307,201308,201309,201310,201311,201312,201401,201402,201403,201404,201405,201406,201407,201408,201409,201410,201411,201412,201501,201502,201503,201504,201505,201506,201507,201508,201509,201510,201511,201512,201601,201602,201603,201604,201605,201606,201607,201608,201609,201610,201611,201612,201701,201702,201703,201704,201705,201706,201707,201708,201709,201710,201711,201712,201801,201802,201803,201804,201805,201806,201807,201808,201809,201810,201811,201812,201901,201902,201903,201904,201905,201906,201907,201908,201909,201910,201911,201912,202001,202002,202003,202004,202005,202006,202007,202008,202009,202010,202011,202012,202101,202102,202103,202104,202105,202106,202107,202108,202109,202110,202111,202112,202201,202202,202203,202204,202205,202206,202207,202208,202209,202210,202211,202212,202301,202302,202303,202304,202305,202306,202307,202308,202309,202310,202311,202312,202401
0,Arson and Criminal Damage,Arson,Barking and Dagenham,6.0,5.0,11.0,10.0,6.0,6.0,13.0,13.0,14.0,5.0,12.0,13.0,13.0,11.0,12.0,10.0,17.0,6.0,15.0,13.0,5.0,10.0,4.0,9.0,4.0,8.0,6.0,4.0,8.0,9.0,6.0,3.0,4.0,2.0,4.0,5.0,7.0,3.0,3.0,6.0,2.0,1.0,5.0,8.0,6.0,5.0,2.0,1.0,5.0,7.0,3.0,6.0,11.0,5.0,5.0,10.0,5.0,3.0,3.0,10.0,5.0,8.0,5.0,11.0,9.0,9.0,5.0,6.0,3.0,5.0,6.0,6.0,5.0,5.0,9.0,5.0,9.0,15.0,3.0,1.0,5.0,5.0,5.0,2.0,13.0,6.0,14.0,2.0,5.0,8.0,7.0,7.0,4.0,2.0,3.0,6.0,3.0,4.0,12.0,6.0,5.0,3.0,8.0,5.0,1.0,5.0,2.0,5.0,5.0,11.0,3.0,5.0,3.0,6.0,9.0,8.0,6.0,4.0,5.0,6.0,2.0,2.0,4.0,4.0,6.0,2.0,7.0,4.0,2.0,4.0,6.0,4.0,6.0,6.0,5.0,4.0,12.0,5.0,7.0,6.0,1.0,4.0,4.0,3.0,5.0,6.0,5.0,9.0,5.0,4.0,4.0,3.0,1.0,3.0,3.0,2.0,2.0,5.0,2.0,5.0,8.0,5.0,4.0,3.0,4.0,4.0
1,Arson and Criminal Damage,Arson,Barnet,11.0,11.0,15.0,18.0,15.0,10.0,10.0,10.0,7.0,3.0,9.0,4.0,11.0,10.0,7.0,3.0,21.0,10.0,9.0,11.0,3.0,4.0,5.0,6.0,3.0,8.0,0.0,7.0,3.0,7.0,11.0,8.0,4.0,1.0,1.0,4.0,2.0,10.0,2.0,6.0,6.0,9.0,5.0,4.0,1.0,2.0,1.0,3.0,5.0,5.0,3.0,5.0,4.0,4.0,4.0,7.0,3.0,4.0,4.0,3.0,5.0,8.0,6.0,8.0,3.0,5.0,9.0,3.0,1.0,2.0,2.0,2.0,14.0,5.0,3.0,5.0,5.0,5.0,7.0,2.0,1.0,5.0,3.0,6.0,4.0,4.0,2.0,6.0,2.0,6.0,9.0,8.0,4.0,4.0,10.0,2.0,5.0,8.0,0.0,6.0,7.0,2.0,6.0,4.0,1.0,1.0,3.0,3.0,6.0,3.0,3.0,3.0,2.0,8.0,6.0,8.0,0.0,5.0,2.0,7.0,7.0,5.0,5.0,0.0,5.0,3.0,5.0,7.0,6.0,3.0,7.0,5.0,5.0,7.0,7.0,2.0,3.0,2.0,6.0,5.0,5.0,8.0,2.0,4.0,2.0,4.0,13.0,8.0,5.0,0.0,6.0,7.0,7.0,4.0,2.0,5.0,8.0,5.0,6.0,7.0,7.0,5.0,11.0,6.0,3.0,7.0
2,Arson and Criminal Damage,Arson,Bexley,12.0,12.0,4.0,10.0,2.0,9.0,8.0,9.0,3.0,3.0,2.0,12.0,5.0,13.0,8.0,3.0,6.0,4.0,3.0,5.0,7.0,5.0,4.0,7.0,7.0,4.0,9.0,5.0,5.0,4.0,1.0,4.0,4.0,6.0,4.0,4.0,3.0,2.0,7.0,2.0,7.0,4.0,2.0,3.0,3.0,1.0,0.0,4.0,4.0,2.0,6.0,3.0,2.0,5.0,3.0,0.0,5.0,2.0,5.0,9.0,12.0,9.0,8.0,7.0,4.0,6.0,15.0,7.0,5.0,3.0,6.0,4.0,1.0,2.0,7.0,2.0,14.0,6.0,10.0,3.0,2.0,10.0,3.0,2.0,6.0,6.0,9.0,5.0,5.0,8.0,14.0,7.0,3.0,4.0,2.0,6.0,3.0,4.0,2.0,7.0,5.0,11.0,4.0,5.0,4.0,8.0,5.0,5.0,8.0,4.0,8.0,4.0,6.0,4.0,8.0,4.0,4.0,2.0,11.0,2.0,9.0,5.0,3.0,6.0,8.0,5.0,6.0,3.0,7.0,1.0,0.0,4.0,7.0,6.0,2.0,5.0,5.0,4.0,6.0,5.0,1.0,3.0,6.0,4.0,6.0,9.0,2.0,4.0,15.0,4.0,2.0,3.0,5.0,1.0,0.0,5.0,5.0,8.0,6.0,9.0,0.0,4.0,7.0,6.0,9.0,11.0
3,Arson and Criminal Damage,Arson,Brent,10.0,6.0,8.0,10.0,8.0,6.0,7.0,10.0,8.0,6.0,3.0,5.0,7.0,5.0,2.0,10.0,17.0,2.0,14.0,11.0,8.0,9.0,4.0,6.0,7.0,3.0,4.0,6.0,4.0,6.0,8.0,3.0,5.0,7.0,0.0,5.0,2.0,3.0,4.0,8.0,7.0,12.0,6.0,4.0,4.0,2.0,5.0,6.0,4.0,6.0,1.0,3.0,3.0,3.0,3.0,13.0,4.0,9.0,2.0,5.0,9.0,3.0,4.0,5.0,5.0,8.0,10.0,4.0,1.0,5.0,11.0,3.0,7.0,4.0,3.0,7.0,7.0,7.0,8.0,4.0,7.0,3.0,0.0,4.0,4.0,3.0,5.0,13.0,8.0,5.0,9.0,7.0,5.0,7.0,17.0,2.0,0.0,2.0,5.0,4.0,6.0,6.0,2.0,14.0,5.0,6.0,3.0,5.0,6.0,5.0,5.0,2.0,8.0,5.0,6.0,3.0,7.0,3.0,1.0,7.0,11.0,4.0,7.0,3.0,8.0,9.0,3.0,7.0,6.0,5.0,7.0,5.0,2.0,11.0,10.0,10.0,5.0,1.0,

MajorText              object
MinorText              object
LookUp_BoroughName     object
201004                float64
201005                float64
                       ...   
202309                float64
202310                float64
202311                float64
202312                float64
202401                float64
Length: 169, dtype: object

Monthly df:
Total nulls: MajorText               0
MinorText               0
LookUp_BoroughName      0
201004                 16
201005                 16
                     ... 
202309                172
202310                172
202311                172
202312                172
202401                172
Length: 169, dtype: int64
Missing rows: 188 out of 1746 (10.77%)
Missing columns: 166 out of 169 (98.22%)


By inspecting the columns the structure appears as:

### Columns
* 'Majortext': This column likely represents a major cateogry of crime.
* 'MinorText': This column represents a more specific subcategory of crime within the major category.
* 'LookUp_BoroughName': This column represents the boroughs in London where the crimes occurred.
* The remaining columns are numerical (floats) and represent the number of incidents for each month, starting from 201004 (April 2010) to 202401 (January 2024).

### Data Types

* The first three columns are of type object likely string
* The remaining month columns are float64. 

The structure of this dataset is suited to a time series analysis.

# Data Pre-processing

In [5]:
# Aggregate data into yearly
non_time_columns = ['MajorText', 'MinorText', 'LookUp_BoroughName']
years = sorted(set(col[:4] for col in df_bronze_crime.columns if col.isdigit()))
df_crime_yearly = aggregate_years(df_bronze_crime, years, non_time_columns)

# Display the first few rows of the yearly data
display(df_crime_yearly.head())

# Missing values analysis for yearly data
print("Yearly df:")
analyse_missing_values(df_crime_yearly)

,MajorText,MinorText,LookUp_BoroughName,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,Arson and Criminal Damage,Arson,Barking and Dagenham,84.0,132.0,75.0,52.0,65.0,77.0,74.0,78.0,58.0,68.0,48.0,66.0,53.0,46.0,4.0
1,Arson and Criminal Damage,Arson,Barnet,107.0,101.0,66.0,51.0,46.0,59.0,53.0,59.0,55.0,46.0,57.0,57.0,66.0,69.0,7.0
2,Arson and Criminal Damage,Arson,Bexley,69.0,71.0,59.0,47.0,35.0,89.0,60.0,78.0,57.0,68.0,67.0,46.0,63.0,60.0,11.0
3,Arson and Criminal Damage,Arson,Brent,73.0,90.0,65.0,62.0,53.0,65.0,73.0,66.0,70.0,61.0,69.0,79.0,67.0,62.0,9.0
4,Arson and Criminal Damage,Arson,Bromley,86.0,131.0,75.0,61.0,80.0,91.0,113.0,103.0,83.0,91.0,112.0,60.0,89.0,83.0,10.0


Yearly df:
Missing rows: 0 out of 1746 (0.00%)
Missing columns: 0 out of 18 (0.00%)


## Impute null values

In [8]:
# Calculate the percentage of missing values for each column
missing_percentage = df_bronze_crime.isnull().sum() / len(df_bronze_crime) * 100

# Sort columns by percentage of missing values
missing_percentage_sorted = missing_percentage.sort_values(ascending=False)

# Display the missing percentages
missing_percentage_sorted

202401                9.851088
202301                9.851088
202202                9.851088
202203                9.851088
202204                9.851088
                        ...   
201501                0.916380
201502                0.916380
LookUp_BoroughName    0.000000
MinorText             0.000000
MajorText             0.000000
Length: 169, dtype: float64

## Imputation
#### The missingness seems to have some uniformity. Since our goal is predictive modelling and the percentage of missing data is relatively low per column (<10%>), for now we will simply impute the mean for the numeric data.

In [13]:
# Simple imputation for missing values
from sklearn.impute import SimpleImputer

# Separating numeric and categorical columns
numeric_cols = df_bronze_crime.select_dtypes(include=['float64']).columns
categorical_cols = df_bronze_crime.select_dtypes(include=['object']).columns

# Impute missing values for numeric columns using median (better robustness against outliers than mean, or you can use mean)
numeric_imputer = SimpleImputer(strategy='median')
df_bronze_crime[numeric_cols] = numeric_imputer.fit_transform(df_bronze_crime[numeric_cols])

# Impute missing values for categorical columns using the most frequent value (mode)
# Note: in this dataset there are no missing values in categorical cols but this is included for completeness and this code can handle changes in future data 
categorical_imputer = SimpleImputer(strategy='most_frequent')
df_bronze_crime[categorical_cols] = categorical_imputer.fit_transform(df_bronze_crime[categorical_cols])

# Verifying that all missing values are imputed
missing_values_after_imputation = df_bronze_crime.isnull().sum().sum()

print(missing_values_after_imputation)


0
